In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from catboost import CatBoostClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import joblib

# Cargar los datos
X = pd.read_csv("../../Data/preprocessed_training_dataset.csv", index_col="respondent_id")
y = pd.read_csv("../../Data/training_set_labels.csv", index_col="respondent_id", usecols=["respondent_id", "h1n1_vaccine", "seasonal_vaccine"])

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [3]:
# Espacio de búsqueda
search_space = {
    'depth': Integer(3, 10),  # Profundidad del árbol
    'learning_rate': Real(0.01, 0.3, prior='log-uniform'),  # Tasa de aprendizaje
    'l2_leaf_reg': Real(1, 10, prior='log-uniform'),  # Regularización L2
    'bagging_temperature': Real(0.0, 1.0),  # Temperatura de bagging
    'iterations': Integer(50, 500),  # Número de iteraciones
    'random_strength': Real(0.0, 10.0),  # Fuerza de aleatoriedad
    'auto_class_weights': Categorical(['Balanced', 'SqrtBalanced']),  # Pesos automáticos
}

In [4]:
# Modelo base para H1N1
catboost_h1n1 = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    verbose=0,
    random_seed=42
)

# BayesSearchCV para H1N1
opt_h1n1 = BayesSearchCV(
    estimator=catboost_h1n1,
    search_spaces=search_space,
    cv=5,
    n_iter=50,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42
)

# Entrenar el modelo para H1N1
opt_h1n1.fit(X_train, y_train['h1n1_vaccine'])

# Mostrar los mejores hiperparámetros para H1N1
print("Mejores hiperparámetros para H1N1:", opt_h1n1.best_params_)
print("Mejor puntaje (AUROC):", opt_h1n1.best_score_)

Mejores hiperparámetros para H1N1: OrderedDict({'auto_class_weights': 'SqrtBalanced', 'bagging_temperature': 0.6326418282192219, 'depth': 5, 'iterations': 383, 'l2_leaf_reg': 9.32484055871332, 'learning_rate': 0.045956717765173294, 'random_strength': 0.41835870030917827})
Mejor puntaje (AUROC): 0.8401836714526072


In [5]:
# Modelo base para vacuna estacional
catboost_seasonal = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    verbose=0,
    random_seed=42
)

# BayesSearchCV para vacuna estacional
opt_seasonal = BayesSearchCV(
    estimator=catboost_seasonal,
    search_spaces=search_space,
    cv=5,
    n_iter=50,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42
)

# Entrenar el modelo para vacuna estacional
opt_seasonal.fit(X_train, y_train['seasonal_vaccine'])

# Mostrar los mejores hiperparámetros para vacuna estacional
print("Mejores hiperparámetros para vacuna estacional:", opt_seasonal.best_params_)
print("Mejor puntaje (AUROC):", opt_seasonal.best_score_)


Mejores hiperparámetros para vacuna estacional: OrderedDict({'auto_class_weights': 'SqrtBalanced', 'bagging_temperature': 0.0, 'depth': 6, 'iterations': 288, 'l2_leaf_reg': 10.0, 'learning_rate': 0.07066138863910175, 'random_strength': 1.0958858256584423})
Mejor puntaje (AUROC): 0.8605428042946428


In [6]:
# Predicciones de probabilidad en el conjunto de prueba
y_proba_h1n1 = opt_h1n1.best_estimator_.predict_proba(X_test)[:, 1]
y_proba_seasonal = opt_seasonal.best_estimator_.predict_proba(X_test)[:, 1]

# Calcular AUROC para cada etiqueta
roc_auc_h1n1 = roc_auc_score(y_test['h1n1_vaccine'], y_proba_h1n1)
roc_auc_seasonal = roc_auc_score(y_test['seasonal_vaccine'], y_proba_seasonal)

print(f"AUROC para H1N1: {roc_auc_h1n1}")
print(f"AUROC para vacuna estacional: {roc_auc_seasonal}")

# Predicciones finales
y_pred_h1n1 = opt_h1n1.best_estimator_.predict(X_test)
y_pred_seasonal = opt_seasonal.best_estimator_.predict(X_test)

# Reporte de clasificación
print("\nReporte de Clasificación para H1N1:")
print(classification_report(y_test['h1n1_vaccine'], y_pred_h1n1))

print("\nReporte de Clasificación para Vacuna Estacional:")
print(classification_report(y_test['seasonal_vaccine'], y_pred_seasonal))

AUROC para H1N1: 0.8456966520033216
AUROC para vacuna estacional: 0.8620802416420121

Reporte de Clasificación para H1N1:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      4207
           1       0.59      0.60      0.60      1135

    accuracy                           0.83      5342
   macro avg       0.74      0.74      0.74      5342
weighted avg       0.83      0.83      0.83      5342


Reporte de Clasificación para Vacuna Estacional:
              precision    recall  f1-score   support

           0       0.80      0.80      0.80      2854
           1       0.77      0.77      0.77      2488

    accuracy                           0.79      5342
   macro avg       0.79      0.79      0.79      5342
weighted avg       0.79      0.79      0.79      5342



In [7]:
# Guardar los mejores modelos
joblib.dump(opt_h1n1.best_estimator_, "CatBoost_H1N1_Model.pkl")
joblib.dump(opt_seasonal.best_estimator_, "CatBoost_Seasonal_Model.pkl")
print("Modelos guardados correctamente.")

Modelos guardados correctamente.


In [8]:
# Cargar el dataset de prueba preprocesado
test_set_preprocessed = pd.read_csv("../../Data/preprocessed_test_dataset.csv")

# Asegurarse de que respondent_id esté disponible
respondent_ids = test_set_preprocessed['respondent_id']

# Eliminar respondent_id del conjunto de características
X_test_final = test_set_preprocessed.drop(columns=['respondent_id'])

# Predicciones de probabilidad para H1N1
y_proba_h1n1_test = opt_h1n1.best_estimator_.predict_proba(X_test_final)[:, 1]  # Probabilidades de la clase positiva (1)

# Predicciones de probabilidad para vacuna estacional
y_proba_seasonal_test = opt_seasonal.best_estimator_.predict_proba(X_test_final)[:, 1]  # Probabilidades de la clase positiva (1)

# Crear el DataFrame de submission
submission = pd.DataFrame({
    "respondent_id": respondent_ids,
    "h1n1_vaccine": y_proba_h1n1_test,
    "seasonal_vaccine": y_proba_seasonal_test
})

# Guardar el archivo de submission
submission.to_csv("Submissions/CatBoost_Doble_submission.csv", index=False)
print("Archivo 'CatBoost_Doble_submission.csv' generado correctamente.")

Archivo 'CatBoost_Optuna_submission.csv' generado correctamente.
